# Find the largest clutser of lipids over time

In lipid membranes, certain lipid species may preferentiall co-localise with one another. The class `lipyphilic.lib.neighbours.Neighbours` can be used to find the largest cluster of specific lipids over time.

Below, we will look at the 58-component plasma memrbane studied by [Ingólfsson et al. (2017)](https://www.cell.com/biophysj/fulltext/S0006-3495(17)31132-3). We will calculate the size of the largest cluster of gangliosides over time.


In [1]:
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import MDAnalysis as mda
from lipyphilic.lib.assign_leaflets import AssignLeaflets
from lipyphilic.lib.neighbours import Neighbours


## Load the topology and trajectory using MDAnalysis

In [2]:
u = mda.Universe("../datafiles/LIM25_neuronal_plasma_membrane.gro")

/home/paul/git-repos/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/paul/git-repos/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/paul/git-repos/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/paul/git-repos/mdanalysis/package/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/paul/git-repos/mdanalysis/

Let's first check that we know how to select all lipids in this complex membrane:

In [4]:
membrane = u.select_atoms("name GL1 GL2 AM1 AM2 ROH").residues

In [5]:
# The above selects all lipids in our system:
print(f"Number of lipids: {membrane.n_residues}")
print(f"Number of lipid species: {np.unique(membrane.resnames).size}")

Number of lipids: 19447
Number of lipid species: 58


## Generate a neighbour matrix

Before we calculate the largest cluster of lipids, we must first use **lipyphilic** to construct an adjacency matrix, $A$, that describes whether each pair of lipid molecules are neighbouring one another or not. The neighbour matrix is defined as follows:

- $A_{ij}=1$ if lipid $i$ is neighbouring lipid $j$
- $A_{ij}=0$ otherwise

**See the [notebook on local lipid environments](4-NeighbourCounts.ipynb) for more information.**


To construct the neighbour matrix, we will use the class `lipyphilic.lib.neighbours.Neighbours`.

In [6]:
# lipid_sel cover all 58 species in the membrane
neighbours = Neighbours(
    universe=u,
    lipid_sel="name GL1 GL2 AM1 AM2 ROH",
    cutoff=12.0
)


We then select which frames of the trajectory to analyse (`None` will use every frame) and select to display a progress bar (`verbose=True`):
  

In [7]:
# Construct the neighbour matrix
neighbours.run(
    start=None,
    stop=None,
    step=None,
    verbose=True
)


  0%|          | 0/1 [00:00<?, ?it/s]

/home/paul/git-repos/mdanalysis/package/MDAnalysis/coordinates/base.py:887: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


## Find the largest cluster of gangliosides

To find the largest cluster of a set of lipid species, we can then use the `largest_cluster` method:

In [13]:
largest_cluster = neighbours.largest_cluster(
    cluster_sel="name GM*"  # selects all gangliosides, which have GM beads in their headgroup
)

  0%|          | 0/1 [00:00<?, ?it/s]

The results are returned in a NumPy array and contain the number of lipids in the largest cluster at each frame


In [15]:
# we only have a single frame in this trajectory
largest_cluster.size

1

At this frame, there are 23 gangliosides in the largest cluster:

In [16]:
largest_cluster

array([23])

# Determine the residue indices of lipids in the largest cluster

If we want to know not just the cluster size but also which lipids are in the largest cluster at each frame, we can set the `return_indices` keyword to `True`:

In [25]:
largest_cluster, largest_cluster_indices = neighbours.largest_cluster(
    cluster_sel="name GM*",
    return_indices=True
)

  0%|          | 0/1 [00:00<?, ?it/s]

The residue indices are returned as list of NumPy array, one per frame of the analysis. Each array contains the residue indices of the lipids in the largest cluster at that frame.

Let's look at indices of lipids in this cluster:


In [26]:
first_frame_indices = largest_cluster_indices[0]

In [28]:
# As expected, there are 23 indices (lipids)
first_frame_indices.size

23

Select the lipids in the largest cluster:

In [29]:
largest_cluster_lipids = u.residues[first_frame_indices]

In [34]:
# Check the species involved in the largest cluster
print(largest_cluster_lipids.resnames)

['DPG1' 'DPG1' 'DPG1' 'DPG1' 'DPG1' 'DPG1' 'DPG1' 'DPG1' 'DPG3' 'DPG3'
 'DPG3' 'DPG3' 'DPG3' 'DPG3' 'DPG3' 'PNG1' 'PNG1' 'DBG1' 'DBG1' 'DBG1'
 'DBG3' 'DBG3' 'DBG3']


# Find the largest cluster in a given leaflet

The previous examples will compute the largest cluster formed by gangliosides at each frame. Depending on the species selected, the largest cluster may span both leaflets of the bilayer.

In order to find the largest cluster at each frame within a given leaflet, we can tell `largest_cluster` to consider only lipids in the upper leaflet by using the `filter_by` parameter.

First, though, we need to know which leaflet each lipid is in at each frame. This may be done using `lipyphilic.lib.assign_leaflets.AssignLeaflets`.

In [36]:
leaflets = AssignLeaflets(
    universe=u,
    lipid_sel="name GL1 GL2 AM1 AM2 ROH",  # select all lipids in the membrane
    n_bins=8  # split the membrane into an 8 by 8 grid, calculate local midpoints for each grid point, and assing leaflets based on local heights
)

In [37]:
# run the analysis on the same frames as Neighbours.run()
leaflets.run(
    start=None,
    stop=None,
    step=None,
    verbose=True
)


  0%|          | 0/1 [00:00<?, ?it/s]

/home/paul/git-repos/mdanalysis/package/MDAnalysis/coordinates/base.py:887: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


The leaflets data are stored in the :attr:`leaflets.leaflets` attribute, will be equal to '1' if the lipid is in the upper leaflet at a given frame and equal to '-1' if it is in the lower leaflet.
 
**See the [notebook on local lipid environments](4-NeighbourCounts.ipynb) for more information.**
  

We can now find the largest cluster over time in the upper leaflet.

The `filter_by` keyword of `largest_cluster` takes as input a two-dimensional NumPy array of shape ($N_{\rm lipids}, N_{\rm frames}$). The array should be a [boolean mask](https://docs.scipy.org/doc/numpy-1.15.0/user/basics.indexing.html#boolean-or-mask-index-arrays), where `True` indicates that we should include this lipid in the neighbour calculation:

In [49]:
# Create a boolean mask
upper_leaflet_mask = leaflets.leaflets == 1

In [50]:
# find the largest cluster over time
largest_cluster_upper_leaflet = neighbours.largest_cluster(
    cluster_sel="name GM*",
    filter_by=upper_leaflet_mask
)


  0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
largest_cluster_upper_leaflet

array([23])

Now, lipids either in the lower leaflet (-1) or the midplane (0) will not be included when determining
the largest cluster.

In this membrane, all gangliosides are in the upper leaflet, so it is expected that the largest cluster size would be unchanged,
